## Miguel Grella
# EDA and Database Normalization for the Fossil Fuels Table

In [1]:
#Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#read the data from remote link
energy_df = pd.read_csv("https://raw.githubusercontent.com/gagaynor/renewable-energy-website/main/raw_data/energy_dataset.csv")



# 1. Transformation for the generation and demand data

In [4]:
#convert time column to datetime and remove time zone
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=True)


#Reformat the time column to only show the date and 24 hour time
energy_df['time'] = energy_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

#convert time column back to datetime and remove time zone
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=False)



In [6]:
#drop columns where all values are missing
energy_df = energy_df.dropna(axis=1, how='all')

#set time as index
energy_df = energy_df.set_index('time', drop=True)

# 2. Organize columns by generation type

In [7]:
#Pass all columns to a list
cols = energy_df.columns.tolist()

#resrict the list to the columns containing 'fossil'
fossil_cols = [col for col in cols if 'fossil' in col]

#Restrict the list to the columns containing 'total' and 'load'
load_cols = [col for col in cols if 'total' in col or 'load' in col]

#Restrict the list to the columns containing 'price'
price_cols = [col for col in cols if 'price' in col]

In [9]:
#Resample main dataframe to daily data
daily_df = energy_df.resample('D').mean()

#Segregate fossil fuel data into a new dataframe

daily_fossil_generation = daily_df[fossil_cols]

In [10]:
#pass columns where all values are zero to a list
zero_cols = daily_fossil_generation.columns[(daily_fossil_generation == 0).all()].tolist()

#restrict data to columns where all values are not zero
daily_fossil_generation = daily_fossil_generation.drop(zero_cols, axis=1)

daily_fossil_generation.head()



,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil
time,,,,
2014-12-31,329.000000,4844.000000,4821.000000,162.000000
2015-01-01,243.708333,4187.791667,4099.458333,161.500000
2015-01-02,0.000000,3566.166667,1258.125000,200.833333
2015-01-03,0.000000,3451.791667,1375.958333,229.791667
2015-01-04,13.583333,3526.125000,2315.291667,356.625000


# 3. Prepare dataframe for database

In [12]:
#change all column names to upper case and remove special characters and spaces
db_cols = daily_fossil_generation.reset_index().columns.str.upper().str.replace('GENERATION FOSSIL', 'GEN').str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')

# Create a new dataframe with the new column names
db_df = daily_fossil_generation.reset_index().copy().rename(columns=dict(zip(daily_fossil_generation.reset_index().columns, db_cols)))

db_df.head()


C:\Users\mikeg\AppData\Local\Temp\ipykernel_9852\3466357894.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  db_cols = daily_fossil_generation.reset_index().columns.str.upper().str.replace('GENERATION FOSSIL', 'GEN').str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')
C:\Users\mikeg\AppData\Local\Temp\ipykernel_9852\3466357894.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  db_cols = daily_fossil_generation.reset_index().columns.str.upper().str.replace('GENERATION FOSSIL', 'GEN').str.replace(' ', '_').str.replace('-', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')


,TIME,GEN_BROWN_COAL_LIGNITE,GEN_GAS,GEN_HARD_COAL,GEN_OIL
0,2014-12-31,329.000000,4844.000000,4821.000000,162.000000
1,2015-01-01,243.708333,4187.791667,4099.458333,161.500000
2,2015-01-02,0.000000,3566.166667,1258.125000,200.833333
3,2015-01-03,0.000000,3451.791667,1375.958333,229.791667
4,2015-01-04,13.583333,3526.125000,2315.291667,356.625000


# The following section  deals with database design

In [13]:
#get path for working directory
path = os.getcwd()

#import database sqliite3 package
import sqlite3

#print *.db files in the working directory
print([file for file in os.listdir(path) if file.endswith('.db')])

['renewable_spain.db']


In [14]:
#connect to database
conn = sqlite3.connect(path + '\\renewable_spain.db')

#Create a cursor
c = conn.cursor()

#store the dataframe in the database, using the table name 'fossil_fuel_generation'
db_df.to_sql('fossil_fuel_generation', conn, if_exists='replace', index=False)


1462

In [15]:
#Check the 'fossil_fuel_generation' table is in the database
pd.read_sql_query("SELECT * FROM fossil_fuel_generation", conn)

,TIME,GEN_BROWN_COAL_LIGNITE,GEN_GAS,GEN_HARD_COAL,GEN_OIL
0,2014-12-31 00:00:00,329.000000,4844.000000,4821.000000,162.000000
1,2015-01-01 00:00:00,243.708333,4187.791667,4099.458333,161.500000
2,2015-01-02 00:00:00,0.000000,3566.166667,1258.125000,200.833333
3,2015-01-03 00:00:00,0.000000,3451.791667,1375.958333,229.791667
4,2015-01-04 00:00:00,13.583333,3526.125000,2315.291667,356.625000
...,...,...,...,...,...
1457,2018-12-27 00:00:00,701.041667,9768.750000,2901.125000,297.000000
1458,2018-12-28 00:00:00,632.000000,7994.166667,2745.625000,273.458333
1459,2018-12-29 00:00:00,0.000000,4972.458333,1716.208333,253.375000
1460,2018-12-30 00:00:00,0.000000,5293.208333,1520.916667,224.083333


In [16]:
#close the connection
conn.close()